In [26]:
import pandas as pd
import numpy as np
import plotly_express as px

## Mungitura

In [92]:
df = pd.read_csv("new_mungitura.csv")

/var/folders/48/tzp62xjn58bb_nw1n30kx0sh0000gn/T/ipykernel_34328/3489801357.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("new_mungitura.csv")


In [66]:
df[['MinutiTotali', 'SecondiTotali']] = df['Durata (MM:SS)'].str.split(':', 1, expand=True)
df['MinutiTotali'] = df['MinutiTotali'].str.strip("0")
df['SecondiTotali'] = df['SecondiTotali'].str.strip("0")

In [93]:
df.loc[df["MinutiTotali"] == ''] = "0"
df.loc[df["SecondiTotali"] == ''] = "0"

In [68]:
df["Produzione PD"]

0         1.67
1         1.67
2         0.84
3         1.26
4         0.03
          ... 
289970    1.51
289971    2.09
289972    1.60
289973    0.00
289974    0.92
Name: Produzione PD, Length: 289975, dtype: float64

In [67]:
df.head()

,Ora inizio,Numero animale,Gruppo,MS/MP,Azione,Durata (MM:SS),Produzione (kg),Numero mungitura,OCC(*1000 Cell/ml),Kick-off,...,Capezzoli non trovatiPD,Capezzoli non trovatiAD,Capezzoli non trovatiPS,Capezzoli non trovatiAS,Kick-offPD,Kick-offAD,Kick-offPS,Kick-offAS,MinutiTotali,SecondiTotali
0,31/08/2020 23:59,173.0,VMS 3,MS3,Mungitura,05:29,5.31,1.0,NaN,NaN,...,0,0,0,0,0,0,0,0,5,29
1,31/08/2020 23:57,935.0,VMS 1,MS1,Mungitura,12:09,3.44,1.0,NaN,NaN,...,0,1,0,0,0,0,0,0,12,9
2,31/08/2020 23:55,21.0,Asciutte,MS2,Mungitura,07:14,2.22,1.0,NaN,NaN,...,0,0,0,0,0,0,0,0,7,14
3,31/08/2020 23:54,38.0,Asciutte,MS3,Mungitura,04:48,2.97,3.0,NaN,NaN,...,0,0,0,0,0,0,0,0,4,48
4,31/08/2020 23:53,536.0,Asciutte,MS4,Mungitura,09:57,0.07,1.0,NaN,NaN,...,1,1,1,1,0,0,0,0,9,57


In [29]:
mask = df["Ora inizio"] == "MANCANO I DATI DAL 17/04/2021 AL 12/02/2021 E NON SONO RECUPERABILI"
i = df[mask].index

In [30]:
df.drop(i,inplace=True)

In [32]:
df['Data'] = pd.to_datetime(df['Ora inizio']).dt.date
df['Tempo'] = pd.to_datetime(df['Ora inizio']).dt.time
df['Mese'] = pd.to_datetime(df['Ora inizio']).dt.month
df['Anno'] = pd.to_datetime(df['Ora inizio']).dt.year
df['Mese Anno'] = [str(x)[0:7] for x in df['Ora inizio']]

In [33]:
columns = ["Produzione (kg)","Flusso AS","Flusso AD","Flusso PS","Flusso PD",
           "Sangue AS","Sangue AD","Sangue PS","Sangue PD",
           "Conducibilità AS","Conducibilità AD","Conducibilità PS","Conducibilità PD",
           "PicchiFlusso AS","PicchiFlusso AD","PicchiFlusso PS","PicchiFlusso PD",
           'Produzione AS', 'Produzione AD','Produzione PS', 'Produzione PD'
]

In [36]:
df["Produzione PD"].str.replace(",",".").astype(float)

0         1.67
1         1.67
2         0.84
3         1.26
4         0.03
          ... 
289971    1.51
289972    2.09
289973    1.60
289974    0.00
289975    0.92
Name: Produzione PD, Length: 289975, dtype: float64

In [37]:
for column in columns:
    df[column] = df[column].astype(str).str.replace(",",".")

In [38]:
df[column] = df[column].astype(float)

In [41]:
df["Incompleto"].unique()

array([nan, 'PS,PD', 'PS', 'PD', 'AS,PS,PD', 'AD,PD', 'AD', 'AS', 'Tutto',
       'AS,AD,PD', 'AS,PD', 'AD,PS,PD', 'AS,PS', 'AS,AD', 'AS,AD,PS',
       'AD,PS'], dtype=object)

In [42]:
df["Capezzolo"].unique()

array([nan, 'PS', 'AD,PS', 'AS', 'AD', 'PD', 'PS,PD', 'Tutto', 'AS,AD',
       'AD,PD', 'AS,PD', 'AS,PS', 'AS,AD,PS', 'AS,PS,PD', 'AS,AD,PD',
       'AD,PS,PD'], dtype=object)

In [43]:
df["Capezzoli non trovati"].unique()

array([nan, 'AS,AD', 'Tutto', 'AS', 'AD', 'PS', 'PD', 'AS,PS', 'AD,PS',
       'AS,AD,PD', 'AD,PD', 'AS,AD,PS', 'AS,PD', 'AS,PS,PD', 'PS,PD',
       'AD,PS,PD'], dtype=object)

In [44]:
df["Kick-off"].unique()

array([nan, 'PD', 'PS', 'PS,PD', 'AS,PS,PD', 'AS', 'AD,PD', 'AS,PS', 'AD',
       'AS,AD', 'AD,PS,PD', 'Tutto', 'AS,AD,PS', 'AS,PD', 'AS,AD,PD',
       'AD,PS'], dtype=object)

In [51]:
base = "Kick-off"
cats = ["PD","AD","AS","PS","Tutto"]
df[base+"PD"]=0
df[base+"AD"]=0
df[base+"PS"]=0
df[base+"AS"]=0

In [ ]:
col = base
for _, i in df.iterrows():
    if str(i[col]) != "nan":
        print(i[col]+" "+str(_))
        if str(i[col]) == "Tutto":
            print("Tutto found "+str(_))
            df.loc[_,base+"AD"] = 1
            df.loc[_,base+"PD"] = 1
            df.loc[_,base+"AS"] = 1
            df.loc[_,base+"PS"] = 1
        else:
            if str(i[base]).find("AD") != -1:
                print("AD found "+str(_))
                df.loc[_,base+"AD"] = 1
            if str(i[base]).find("PD")!= -1:
                print("PD found "+str(_))
                df.loc[_,base+"PD"] = 1
            if str(i[base]).find("PS")!= -1:
                print("PS found "+str(_))
                df.loc[_,base+"PS"] = 1
            if str(i[base]).find("AS")!= -1:
                print("AS found "+str(_))
                df.loc[_,base+"AD"] = 1

In [56]:
df.head()

,Ora inizio,Numero animale,Gruppo,MS/MP,Azione,Durata (MM:SS),Produzione (kg),Numero mungitura,OCC(*1000 Cell/ml),Kick-off,...,CapezzoloPS,CapezzoloAS,Capezzoli non trovatiPD,Capezzoli non trovatiAD,Capezzoli non trovatiPS,Capezzoli non trovatiAS,Kick-offPD,Kick-offAD,Kick-offPS,Kick-offAS
0,31/08/2020 23:59,173.0,VMS 3,MS3,Mungitura,05:29,5.31,1.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,31/08/2020 23:57,935.0,VMS 1,MS1,Mungitura,12:09,3.44,1.0,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0
2,31/08/2020 23:55,21.0,Asciutte,MS2,Mungitura,07:14,2.22,1.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,31/08/2020 23:54,38.0,Asciutte,MS3,Mungitura,04:48,2.97,3.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,31/08/2020 23:53,536.0,Asciutte,MS4,Mungitura,09:57,0.07,1.0,NaN,NaN,...,0,0,1,1,1,1,0,0,0,0


In [95]:
df.to_csv("new_mungitura.csv",index=False)

## Traffico

In [76]:
df = pd.read_csv("new_traffico_animali.csv")

/var/folders/48/tzp62xjn58bb_nw1n30kx0sh0000gn/T/ipykernel_34328/354719305.py:1: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("new_traffico_animali.csv")


In [89]:
df.head()

,Numero animale,Nome gruppo,Data/Ora,Area precedente,Tempo nell'area (hh:mm),Nome dispositivo,Risultato,Decisione cancello,Intervallo Mungitura,Capezzoli non completamente munti,...,Tempo,Mese,Anno,Mese Anno,PD,AD,PS,AS,OreTotali,MinutiTotali
0,980,Asciutte,21/03/2022 08:46,MS1,06:26,MS1,NaN,NaN,NaN,NaN,...,08:46:00,3.0,2022.0,2022-03,0,0,0,0,6,26
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,980,Asciutte,20/03/2022 02:07,MS1,14:12,MS1,0.05,NaN,1 d 14:12,NaN,...,02:07:00,3.0,2022.0,2022-03,0,0,0,0,14,12
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,980,Asciutte,18/03/2022 11:52,MS1,06:12,MS1,0.01,NaN,06:12,NaN,...,11:52:00,3.0,2022.0,2022-03,0,0,0,0,6,12


In [81]:
df["OreTotali"].fill

In [78]:
col = "Tempo nell'area (hh:mm)"

df[['OreTotali', 'MinutiTotali']] = df[col].str.split(':', 1, expand=True)
df['MinutiTotali'] = df['MinutiTotali'].str.strip("0")
df['OreTotali'] = df['OreTotali'].str.strip("0")

In [87]:
df.loc[df["MinutiTotali"] == ''] = "0"
df.loc[df["OreTotali"] == ''] = "0"

In [ ]:
df['Data'] = pd.to_datetime(df['Data/Ora']).dt.date

In [ ]:
df['Tempo'] = pd.to_datetime(df['Data/Ora']).dt.time
df['Mese'] = pd.to_datetime(df['Data/Ora']).dt.month
df['Anno'] = pd.to_datetime(df['Data/Ora']).dt.year
df['Mese Anno'] = [str(x)[0:7] for x in df['Data']]

In [ ]:
mask = df["Risultato"].str.contains("kg",na=False)

In [ ]:
df["Risultato"] = np.where(mask,df["Risultato"].str.replace(",","."),0)

In [ ]:
df["Risultato"] = df["Risultato"].astype('float')

In [ ]:
df['Risultato'] = df['Risultato'].str.extract('([0-9][,.]*[0-9]*)')

In [5]:
cats = ["PD","AD","AS","PS","Tutto"]
df["PD"]=0
df["AD"]=0
df["PS"]=0
df["AS"]=0

In [15]:
col = "Capezzoli non completamente munti"
for _, i in df.iterrows():
    if str(i[col]) != "nan":
        print(i[col]+" "+str(_))
        if str(i[col]) == "Tutto":
            print("Tutto found "+str(_))
            df.loc[_,"AD"] = 1
            df.loc[_,"PD"] = 1
            df.loc[_,"AS"] = 1
            df.loc[_,"PS"] = 1
        else:
            if str(i["Capezzoli non completamente munti"]).find("AD") != -1:
                print("AD found "+str(_))
                df.loc[_,"AD"] = 1
            if str(i["Capezzoli non completamente munti"]).find("PD")!= -1:
                print("PD found "+str(_))
                df.loc[_,"PD"] = 1
            if str(i["Capezzoli non completamente munti"]).find("PS")!= -1:
                print("PS found "+str(_))
                df.loc[_,"PS"] = 1
            if str(i["Capezzoli non completamente munti"]).find("AS")!= -1:
                print("AS found "+str(_))
                df.loc[_,"AD"] = 1

PD 39
PD found 39
PD 43
PD found 43
PD 51
PD found 51
PD 59
PD found 59
PD 61
PD found 61
PD 65
PD found 65
PD 77
PD found 77
PD 87
PD found 87
AD,PD 93
AD found 93
PD found 93
AD,PD 103
AD found 103
PD found 103
AD 116
AD found 116
AD 120
AD found 120
PD 136
PD found 136
AS,AD,PD 140
AD found 140
PD found 140
AS found 140
PD 150
PD found 150
AD,PD 154
AD found 154
PD found 154
PD 166
PD found 166
PD 170
PD found 170
AS,AD,PD 172
AD found 172
PD found 172
AS found 172
PD 198
PD found 198
AS,AD,PD 211
AD found 211
PD found 211
AS found 211
PD 225
PD found 225
AD,PD 227
AD found 227
PD found 227
AS,PD 249
PD found 249
AS found 249
PD 261
PD found 261
AD,PD 263
AD found 263
PD found 263
PD 273
PD found 273
PD 275
PD found 275
PD 293
PD found 293
AS,AD,PD 304
AD found 304
PD found 304
AS found 304
PD 322
PD found 322
PD 326
PD found 326
PD 350
PD found 350
PD 355
PD found 355
PD 383
PD found 383
PD 385
PD found 385
AS,PD 413
PD found 413
AS found 413
AS,AD 429
AD found 429
AS found 429
PD 

In [23]:
df.describe()

,Risultato,Mese,Anno,PD,AD,PS,AS
count,260667.000000,872742.000000,872742.000000,872750.000000,872750.000000,872750.000000,872750.000000
mean,3.937369,6.240480,2021.118821,0.010160,0.002806,0.010221,0.000445
std,1.896292,3.579812,0.765761,0.100283,0.052898,0.100579,0.021080
min,0.000000,1.000000,2020.000000,0.000000,0.000000,0.000000,0.000000
25%,2.750000,3.000000,2021.000000,0.000000,0.000000,0.000000,0.000000
50%,3.770000,6.000000,2021.000000,0.000000,0.000000,0.000000,0.000000
75%,4.990000,9.000000,2022.000000,0.000000,0.000000,0.000000,0.000000
max,26.000000,12.000000,2022.000000,1.000000,1.000000,1.000000,1.000000


In [90]:
df.to_csv("new_traffico_animali.csv",index=False)

## Mungitura eventi

In [115]:
mungitura_eventi = pd.read_csv("MungituraEventi.csv")

In [116]:
mungitura_eventi['Data'] = pd.to_datetime(mungitura_eventi['Data evento']).dt.date
mungitura_eventi['Tempo'] = pd.to_datetime(mungitura_eventi['Data evento']).dt.time
mungitura_eventi['Mese'] = pd.to_datetime(mungitura_eventi['Data evento']).dt.month
mungitura_eventi['Anno'] = pd.to_datetime(mungitura_eventi['Data evento']).dt.year

In [117]:
mungitura_eventi['Mese Anno'] = [str(x)[0:7] for x in mungitura_eventi['Data']]

## Istogramma eventi

In [84]:
px.histogram(mungitura_eventi,x="Tipo")

In [85]:
px.pie(mungitura_eventi,names="Tipo")

## Serie temporale x=data y=tipo

In [118]:
mungitura_eventi["id"] = range(mungitura_eventi.shape[0])

In [119]:
mungitura_eventi["count"] = mungitura_eventi.groupby(["Mese Anno"])["id"].transform("count")

In [92]:
px.histogram(mungitura_eventi,x="Mese Anno",color="Tipo")

In [95]:
mungitura_eventi.shape

(10422, 14)

In [97]:
fig = px.line(mungitura_eventi, x="Mese Anno",y="count", color="Tipo")
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")
fig.show()

In [121]:
mungitura_eventi.to_csv("new_mungitura_eventi.csv",index=False)